In [98]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:
import anndata
import numpy as np
import pickle
from moscot.framework.estimators import LineageEstimator

In [100]:
adata_dir = "/Users/DominikKlein/Documents/PhD/data/Hu_zebrafish_linnaeus/adata.h5ad"
trees_dir = "/Users/DominikKlein/Documents/PhD/data/Hu_zebrafish_linnaeus/trees.pkl"

In [101]:
adata = anndata.read(adata_dir)

In [102]:
# focus on time-points ctrl 3, 7 (H5-> ctrl, Hr26 -> 3, Hr2->7)
dpis_orig_ident = ['H5', 'Hr26', 'Hr2a','Hr2b']
dpis = [0, 3, 7, 7]
dpis_unq = np.unique(dpis)
dpis_str =['ctrl', 'dpi3', 'dpi7']
dpis_trees = ['H5', 'Hr26', 'Hr2']
core_nodes = [100, 100, 100]

In [103]:
adata_filtered = adata[adata.obs['orig.ident'].isin(dpis_orig_ident)].copy()
adata_filtered

AnnData object with n_obs × n_vars = 49249 × 31466
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mito', 'time', 'morphine', 'AV', 'inhib', 'so.far.ident.subtypes', 'so.far.ident', 'RNA_snn_res.4', 'seurat_clusters', 'work.ident', 'fibro.subtypes', 'is.inhib', 'split.inhib', 'zoom.ident', 'zoom.subtypes', 'plot.ident', 'CM.ident', 'paper.ident', 'RNA_snn_res.2', 'RNA_snn_res.3', 'RNA_snn_res.3.5', 'work.ident2', 'work.ident3', 'final.zoom', 'lineage.ident', 'Cell_type'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    uns: 'trees'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [104]:
inds = np.random.randint(0, adata_filtered.shape[0], 1000)
adata_filtered = adata_filtered[inds].copy()

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [105]:
adata_filtered

AnnData object with n_obs × n_vars = 1000 × 31466
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mito', 'time', 'morphine', 'AV', 'inhib', 'so.far.ident.subtypes', 'so.far.ident', 'RNA_snn_res.4', 'seurat_clusters', 'work.ident', 'fibro.subtypes', 'is.inhib', 'split.inhib', 'zoom.ident', 'zoom.subtypes', 'plot.ident', 'CM.ident', 'paper.ident', 'RNA_snn_res.2', 'RNA_snn_res.3', 'RNA_snn_res.3.5', 'work.ident2', 'work.ident3', 'final.zoom', 'lineage.ident', 'Cell_type'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    uns: 'trees'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [106]:
adata_filtered.obs["dpi"] = adata_filtered.obs["orig.ident"].replace(dpis_orig_ident, dpis).astype("category")

In [107]:
with open(trees_dir, 'rb') as file:
    trees = pickle.load(file)

In [108]:
trees_dpis = {}
trees_dpis_core = {}
for i, dpi in enumerate(dpis_unq):
    trees_dpis[dpi] = trees[dpis_trees[i]]
    trees_dpis_core[dpi] = trees_dpis[dpi].subgraph([str(n) for n in range(core_nodes[i])])


In [109]:
key = "dpi"
epsilon = 0.2

In [110]:
estimator = LineageEstimator(adata_filtered, "dpi", trees_dpis_core, epsilon)

In [111]:
estimator.prepare((0,3))

In [112]:
estimator.geometries_intra_dict[0].cost_matrix.shape, estimator.geometries_intra_dict[3].cost_matrix.shape

((17, 17), (16, 16))

In [115]:
estimator.geometries_inter_dict[(0,3)]

In [116]:
cost_fn = Euclidean()
from ott.geometry import PointCloud
rep = "X"
transport_tuple = (0,3)
inter_pc = PointCloud(
            getattr(adata_filtered[adata_filtered.obs[key] == transport_tuple[0]], rep).todense(),
            getattr(adata_filtered[adata_filtered.obs[key] == transport_tuple[1]], rep).todense(),
            cost_fn=cost_fn,
            online=online
        )

In [117]:
inter_pc.cost_matrix

In [119]:
getattr(adata_filtered[adata_filtered.obs[key] == transport_tuple[1]], rep).todense().shape

(267, 31466)

In [121]:
adata_filtered[adata_filtered.obs[key] == transport_tuple[1]].obs[key].unique()

[3]
Categories (1, int64): [3]

In [125]:
len(trees["H5"].nodes)

1055

In [55]:
getattr(adata_filtered[adata_filtered.obs[key] == transport_tuple[0]], rep).todense()

matrix([[13.,  0.,  0., ..., 73.,  0.,  0.],
        [ 1.,  0.,  0., ...,  9.,  0.,  0.],
        [ 3.,  0.,  0., ..., 15.,  0.,  0.],
        ...,
        [49.,  0.,  0., ..., 42.,  0.,  0.],
        [ 4.,  0.,  0., ..., 23.,  0.,  0.],
        [ 1.,  0.,  0., ..., 58.,  0.,  0.]], dtype=float32)

In [57]:
from scipy.sparse import issparse
issparse(getattr(adata_filtered, rep))

True

In [53]:
adata_filtered[adata_filtered.obs[key] == transport_tuple[1]]

View of AnnData object with n_obs × n_vars = 290 × 31466
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mito', 'time', 'morphine', 'AV', 'inhib', 'so.far.ident.subtypes', 'so.far.ident', 'RNA_snn_res.4', 'seurat_clusters', 'work.ident', 'fibro.subtypes', 'is.inhib', 'split.inhib', 'zoom.ident', 'zoom.subtypes', 'plot.ident', 'CM.ident', 'paper.ident', 'RNA_snn_res.2', 'RNA_snn_res.3', 'RNA_snn_res.3.5', 'work.ident2', 'work.ident3', 'final.zoom', 'lineage.ident', 'Cell_type', 'dpi'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    uns: 'trees'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [51]:
inter_pc.cost_matrix

In [45]:
dir(inter_pc)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply_cost_to_vec',
 '_apply_transport_from_potentials',
 '_apply_transport_from_scalings',
 '_axis_norm',
 '_center',
 '_cost_fn',
 '_cost_matrix',
 '_epsilon',
 '_epsilon_init',
 '_kernel_matrix',
 '_kwargs',
 '_norm_x',
 '_norm_y',
 '_online',
 '_relative_epsilon',
 '_scale',
 '_softmax',
 'apply_cost',
 'apply_kernel',
 'apply_lse_kernel',
 'apply_transport_from_potentials',
 'apply_transport_from_scalings',
 'copy_epsilon',
 'cost_matrix',
 'epsilon',
 'error',
 'is_symmetric',
 'kernel_matrix',
 'marginal_from_potentials',
 'marginal_from_scalings',
 'mean_cost_matrix',
 'median_cost_matrix',
 'potential_from_scal

In [27]:
estimator.geometries_inter_dict[(0, 3)].cost_matrix.shape

LinAlgError: Last 2 dimensions of the array must be square

In [36]:
ot_results = estimator.fit()

LinAlgError: Last 2 dimensions of the array must be square

In [ ]:
self._solver_dict[tup].fit(self.geometries_inter_dict[tup], self.geometries_intra_dict[tup[0]], 
                           self.geometries_intra_dict[tup[1]], self.a_dict[tup], self.b_dict[tup])